## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import os


## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [2]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [3]:
df.info()

In [4]:
df.drop(columns='ID').describe()

The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

In [5]:
for col in df.columns:
    print(df[col].value_counts())
    print('////////////////////////////\n')

In [6]:
df.head()

### Read Weather CSV

In [7]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

weather_df.head()

In [8]:
weather_df.isna().sum()

In [9]:
print(weather_df.head())

weather_df = weather_df.drop(columns=['Wind_Chill(F)'])
weather_df['Precipitation(in)'] = weather_df['Precipitation(in)'].fillna(weather_df['Precipitation(in)'].median())
weather_df['Wind_Speed(mph)'] = weather_df['Wind_Speed(mph)'].fillna(weather_df['Wind_Speed(mph)'].median())
weather_df.isna().sum()

In [10]:
weather_df = weather_df.dropna()
weather_df.isna().sum()

In [11]:
print(weather_df.shape)

In [12]:
weather_df['Date'] = weather_df['Year']*10000 + weather_df['Month']*100 + weather_df['Day']
weather_df.head()

In [13]:
weather_df = weather_df.rename(columns={'Hour': 'Time'})
weather_df = weather_df.drop(columns=['Year', 'Day', 'Month'])
weather_df.head()

In [14]:
weather_df['Weather_Condition'] = weather_df['Weather_Condition'].astype('category')
weather_df['Weather_Condition'] = weather_df['Weather_Condition'].cat.codes
weather_df.head()

In [15]:
weather_df['datetime'] = weather_df['Date'] * 100 + weather_df['Time']
weather_df.head()

In [16]:
print(weather_df.shape)

In [17]:
weather_df = weather_df.drop_duplicates(subset='datetime', keep='last')
weather_df.head()

In [18]:
print(weather_df.shape)

### Feature Manipulation

In [19]:
cols_to_encode = ['Give_Way' , 'No_Exit','Crossing', 'Junction', 'Railway', 'Stop', 'Amenity', 'Side']
for col in cols_to_encode:
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.codes
    

df.head()

In [20]:
df[['Date', 'Time']] = df['timestamp'].str.split(' ', expand=True)
df['Date'] = df['Date'].apply(lambda x: x[0:4] + x[5:7] + x[8:])
df['Time'] = df['Time'].apply(lambda x: x[0:2])
df['Date'] = df['Date'].astype('int')
df['Time'] = df['Time'].astype('int')

In [21]:
df = df.drop(columns=['Bump', 'Roundabout'])
df.head()

In [22]:
#Remove Timestamp
df = df.drop(columns=['timestamp'])
df.head()

### Merge Crash and Weather Datasets

In [23]:
df = pd.merge(df, weather_df, on=['Date', 'Time'], how='left')

In [24]:
print(df.isna().sum())

In [25]:
df = df.dropna()

In [26]:
print(df.shape)

In [27]:
df.head()

In [28]:
df = df.drop(columns=['datetime', 'Selected'])

### Read holidays file and convert it to csv file

In [29]:
from bs4 import BeautifulSoup
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

file = open(os.path.join(dataset_path, 'holidays.xml'))
contents = file.read()
soup = BeautifulSoup(contents, 'xml')

date = soup.find_all('date')
description = soup.find_all('description')

data = []
for i in range(0, len(date)):
    rows = [date[i].get_text(), description[i].get_text()]
    data.append(rows)

x_df = pd.DataFrame(data, columns=['date','description'], dtype = float)
print(x_df)    


In [30]:
x_df['date'] = x_df['date'].apply(lambda x: x[0:4] + x[5:7] + x[8:])
print(x_df)

In [31]:
x_df.head()

In [32]:
#Try Merge and add description field
x_df.rename(columns={'date': 'Date'}, inplace=True)
x_df['Date'] = x_df['Date'].astype('int')
df = pd.merge(df, x_df, on='Date', how='left')
df.head()

In [33]:
df.isna().sum()

In [34]:
df['description'] = df['description'].fillna('None')

In [35]:
df.head()

In [36]:
df['description'].value_counts()

In [37]:
df['description'] = df['description'].astype('category')
df['description'] = df['description'].cat.codes
df.head()

In [38]:
#print(df['holiday'].value_counts())

In [39]:
df.head()

#### Feature Removal

In [40]:
print(df.info())

In [41]:
df = df.drop(columns=['Humidity(%)', 'Amenity'])

In [42]:
print(df.info())

In [43]:
df.head()

In [44]:
import math
df['x'] = df['Lat'].apply(math.cos) * df['Lng'].apply(math.cos)
df['y'] = df['Lat'].apply(math.cos) * df['Lng'].apply(math.sin)
df['z'] = df['Lat'].apply(math.sin)
df = df.drop(columns=['Lat', 'Lng'])
df.head()

In [45]:
df.info()

In [46]:
#Normalize Features
for col in df.columns:
    if col not in ['ID', 'Severity', 'x', 'y', 'z', 'Date', 'Time']:
        df[col] = (df[col] - df[col].mean()) / df[col].std()
df.head()

#### Add Day_Of_Week column

In [47]:
import datetime

In [48]:
def get_day_of_week(date):
    date = int(date)
    day = date%100
    date = date//100
    month = date%100
    year = date//100
    return datetime.datetime(year, month, day).weekday()

In [49]:
df['DayOfWeek'] = df['Date'].apply(get_day_of_week)
df.head()

#### Get Time of Day

In [50]:
def get_time_of_day(time):
    if time >= 0 and time < 6:
        return 0
    if time >= 6 and time < 12:
        return 1
    if time >= 12 and time < 18:
        return 2
    return 3

In [51]:
df['TimeOfDay'] = df['Time'].apply(get_time_of_day)
df.head()

In [52]:
df = df.drop(columns='Time')

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [53]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Severity']) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID','Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID' ,'Severity'])
y_val = val_df['Severity']


As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [54]:
#Checking for duplicates
duplicates = X_train[X_train.duplicated()]
print(duplicates)

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [55]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [56]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

In [57]:
#Train with all data
X_train = df.drop(columns=['ID', 'Severity'])
y_train = df['Severity']

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [58]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))

Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [59]:
X_test = test_df.drop(columns=['ID'])
print(X_test)

# You should update/remove the next line once you change the features used for training
#X_test = X_test[['Lat', 'Lng', 'Distance(mi)']]
for col in cols_to_encode:
    X_test[col] = X_test[col].astype('category')
    X_test[col] = X_test[col].cat.codes
X_test = X_test.drop(columns=['Bump', 'Roundabout'])

X_test[['Date', 'Time']] = X_test['timestamp'].str.split(' ', expand=True)
X_test['Date'] = X_test['Date'].apply(lambda x: x[0:4] + x[5:7] + x[8:])
X_test['Time'] = X_test['Time'].apply(lambda x: x[0:2])
X_test['Date'] = X_test['Date'].astype('int')
X_test['Time'] = X_test['Time'].astype('int')
X_test = X_test.drop(columns=['timestamp'])


X_test = pd.merge(X_test, weather_df, on=['Date', 'Time'], how='left')

X_test = X_test.dropna()
X_test = X_test.drop(columns='datetime')


x_df = x_df.rename(columns={'date': 'Date'})
x_df['Date'] = x_df['Date'].astype('int')

#mask = (X_test.set_index([ 'Date' ]).index.isin(x_df.set_index([ 'Date' ]).index))

#X_test['holiday'] = np.where(mask, 1, 0)
X_test = pd.merge(X_test, x_df, on='Date', how='left')
#X_test.head()
X_test['description'] = X_test['description'].fillna('None')
X_test['description'] = X_test['description'].astype('category')
X_test['description'] = X_test['description'].cat.codes

X_test = X_test.drop(columns=['Humidity(%)', 'Amenity', 'Selected'])

X_test['DayOfWeek'] = X_test['Date'].apply(get_day_of_week)
X_test['TimeOfDay'] = X_test['Time'].apply(get_time_of_day)
X_test = X_test.drop(columns='Time')


X_test['x'] = X_test['Lat'].apply(math.cos) * X_test['Lng'].apply(math.cos)
X_test['y'] = X_test['Lat'].apply(math.cos) * X_test['Lng'].apply(math.sin)
X_test['z'] = X_test['Lat'].apply(math.sin)
X_test = X_test.drop(columns=['Lat', 'Lng'])

print(X_test.info())

In [60]:
y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [61]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.